In [1]:
import json
from dataclasses import dataclass
from typing import Optional

@dataclass
class PatientData:
    question2: Optional[int] = None  # Age
    question16: Optional[str] = None  # Personal history of CRC or polyp
    question17: Optional[str] = None  # Family history of CRC or polyp
    question18: Optional[int] = None  # Age of family member diagnosis (if family history positive)


def analyze_colorectal_cancer_screening(patient_data: PatientData) -> str:
    """
    Analyzes patient data for colorectal cancer screening indication based on average and enhanced risk guidelines.

    Args:
        patient_data: PatientData object containing answers to screening questions.

    Returns:
        A string indicating screening recommendation or reasons for no indication, including enhanced guidelines for family history.
    """

    # Check for missing required questions
    if patient_data.question2 is None:
        return "Missing required information: Please provide your age (question2)."
    if patient_data.question16 is None:
        return "Missing required information: Please answer if you have a personal history of CRC or advanced polyp (question16)."
    if patient_data.question17 is None:
        return "Missing required information: Please answer if you have a family history of CRC or advanced polyp (question17)."

    age = patient_data.question2
    personal_history = patient_data.question16
    family_history = patient_data.question17
    family_diagnosis_age = patient_data.question18

    # Check for personal history of CRC or advanced polyp (NOT average risk)
    if personal_history == "Item 2":  # "بله" - Yes
        return "Patient is not indicated for colorectal cancer screening as per average risk guidelines due to personal history of CRC or advanced polyp. Further evaluation is needed based on personal history."

    # Check for family history of CRC or advanced polyp (Enhanced screening)
    if family_history == "Item 2":  # "بله" - Yes
        if family_diagnosis_age is None:
            return "Missing required information: Please provide the age of diagnosis of the family member with CRC or advanced polyp (question18) as you indicated a family history."

        start_screening_age = max(40, family_diagnosis_age - 10)

        if age >= start_screening_age and age <= 85: # Extend screening age for family history up to 85 based on guidelines.
            return f"Patient is indicated for colorectal cancer screening due to family history. Enhanced screening recommended. Start screening at age {start_screening_age}. Recommended modality: Colonoscopy, Frequency: Every 5 years. If colonoscopy is declined, offer annual Fecal Immunochemical Test (FIT) or multitarget stool DNA testing (mt-sDNA) every 3 years."
        elif age < start_screening_age:
            return f"Patient is not yet indicated for colorectal cancer screening based on age. Enhanced screening recommended to start at age {start_screening_age} due to family history."
        elif age > 85:
             return "Patient is not indicated for routine colorectal cancer screening based on age criteria for enhanced risk individuals over 85 years old. Decision to screen should be individualized based on life expectancy and prior screening history."


    elif family_history == "Item 3": # "نمی دانم / بررسی نکرده ام" - Don't know/Not checked
        recommendation = "Family history of colorectal cancer or advanced polyps is unknown and should be investigated to determine long-term risk and screening strategy.  For the present, using average risk guidelines: "
        if 45 <= age <= 75:
            recommendation += "Patient is indicated for colorectal cancer screening as average risk. Recommended modality: Colonoscopy, Frequency: Every 10 years or Fecal Immunochemical Test (FIT), Frequency: Annually."
        elif age < 45:
            recommendation += "Patient is not indicated for colorectal cancer screening based on age criteria for average risk individuals under 45 years old."
        elif age > 75:
            recommendation += "Patient is not indicated for routine colorectal cancer screening based on age criteria for average risk individuals over 75 years old. Decision to screen should be individualized based on life expectancy and prior screening history."
        else:
            recommendation += "Patient is not indicated for colorectal cancer screening based on current guidelines for average risk individuals."
        return recommendation


    # Average risk assessment for age 45-75 if no personal or family history flags
    if 45 <= age <= 75:
        return "Patient is indicated for colorectal cancer screening as average risk. Recommended modality: Colonoscopy, Frequency: Every 10 years or Fecal Immunochemical Test (FIT), Frequency: Annually."
    elif age < 45:
        return "Patient is not indicated for colorectal cancer screening based on age criteria for average risk individuals under 45 years old, and no family history of CRC or advanced polyp reported to suggest enhanced screening at a younger age."
    elif age > 75:
        return "Patient is not indicated for routine colorectal cancer screening based on age criteria for average risk individuals over 75 years old.  Decision to screen should be individualized based on life expectancy and prior screening history."
    else:
        return "Patient is not indicated for colorectal cancer screening based on current guidelines for average risk individuals."


def read_json_from_file(file_path: str) -> PatientData:
    """
    Reads patient data from a JSON file and returns a PatientData object.
    """
    with open(file_path, 'r', encoding='utf-8') as f:  # Specify encoding if needed
        data = json.load(f)

    # Assuming the JSON structure matches the question names directly
    patient_data = PatientData(
        question2=data.get("question2"),
        question16=data.get("question16"),
        question17=data.get("question17"),
        question18=data.get("question18")
    )
    return patient_data


if __name__ == "__main__":
    # Example Usage - No file reading, using PatientData objects directly

    # Example 1: Average risk, age 50
    patient_data_avg_risk_50 = PatientData(question2=50, question16="Item 1", question17="Item 1")
    print(f"Example 1 (Average risk, age 50): {analyze_colorectal_cancer_screening(patient_data_avg_risk_50)}")

    # Example 2: Average risk, age 30
    patient_data_avg_risk_30 = PatientData(question2=30, question16="Item 1", question17="Item 1")
    print(f"Example 2 (Average risk, age 30): {analyze_colorectal_cancer_screening(patient_data_avg_risk_30)}")

    # Example 3: Average risk, age 80
    patient_data_avg_risk_80 = PatientData(question2=80, question16="Item 1", question17="Item 1")
    print(f"Example 3 (Average risk, age 80): {analyze_colorectal_cancer_screening(patient_data_avg_risk_80)}")

    # Example 4: Personal history
    patient_data_personal_history = PatientData(question2=60, question16="Item 2", question17="Item 1")
    print(f"Example 4 (Personal history): {analyze_colorectal_cancer_screening(patient_data_personal_history)}")

    # Example 5: Family history, age 50, family diagnosis 45
    patient_data_family_history_50 = PatientData(question2=50, question16="Item 1", question17="Item 2", question18=45)
    print(f"Example 5 (Family history, age 50, family dx 45): {analyze_colorectal_cancer_screening(patient_data_family_history_50)}")

    # Example 6: Family history, age 35, family diagnosis 45
    patient_data_family_history_35 = PatientData(question2=35, question16="Item 1", question17="Item 2", question18=45)
    print(f"Example 6 (Family history, age 35, family dx 45): {analyze_colorectal_cancer_screening(patient_data_family_history_35)}")

    # Example 7: Family history, age 80, family diagnosis 60
    patient_data_family_history_80 = PatientData(question2=80, question16="Item 1", question17="Item 2", question18=60)
    print(f"Example 7 (Family history, age 80, family dx 60): {analyze_colorectal_cancer_screening(patient_data_family_history_80)}")

    # Example 8: Uncertain family history, age 50
    patient_data_uncertain_family_50 = PatientData(question2=50, question16="Item 1", question17="Item 3")
    print(f"Example 8 (Uncertain family history, age 50): {analyze_colorectal_cancer_screening(patient_data_uncertain_family_50)}")

    # Example 9: Uncertain family history, age 30
    patient_data_uncertain_family_30 = PatientData(question2=30, question16="Item 1", question17="Item 3")
    print(f"Example 9 (Uncertain family history, age 30): {analyze_colorectal_cancer_screening(patient_data_uncertain_family_30)}")

    # Example 10: Uncertain family history, age 80
    patient_data_uncertain_family_80 = PatientData(question2=80, question16="Item 1", question17="Item 3")
    print(f"Example 10 (Uncertain family history, age 80): {analyze_colorectal_cancer_screening(patient_data_uncertain_family_80)}")

    # Example 11: Missing age
    patient_data_missing_age = PatientData(question16="Item 1", question17="Item 1")
    print(f"Example 11 (Missing age): {analyze_colorectal_cancer_screening(patient_data_missing_age)}")

    # Example 12: Missing personal history
    patient_data_missing_personal_history = PatientData(question2=50, question17="Item 1")
    print(f"Example 12 (Missing personal history): {analyze_colorectal_cancer_screening(patient_data_missing_personal_history)}")

    # Example 13: Missing family history
    patient_data_missing_family_history = PatientData(question2=50, question16="Item 1")
    print(f"Example 13 (Missing family history): {analyze_colorectal_cancer_screening(patient_data_missing_family_history)}")

    # Example 14: Missing family diagnosis age (when family history is positive)
    patient_data_missing_family_diag_age = PatientData(question2=50, question16="Item 1", question17="Item 2")
    print(f"Example 14 (Missing family diagnosis age): {analyze_colorectal_cancer_screening(patient_data_missing_family_diag_age)}")

Example 1 (Average risk, age 50): Patient is indicated for colorectal cancer screening as average risk. Recommended modality: Colonoscopy, Frequency: Every 10 years or Fecal Immunochemical Test (FIT), Frequency: Annually.
Example 2 (Average risk, age 30): Patient is not indicated for colorectal cancer screening based on age criteria for average risk individuals under 45 years old, and no family history of CRC or advanced polyp reported to suggest enhanced screening at a younger age.
Example 3 (Average risk, age 80): Patient is not indicated for routine colorectal cancer screening based on age criteria for average risk individuals over 75 years old.  Decision to screen should be individualized based on life expectancy and prior screening history.
Example 4 (Personal history): Patient is not indicated for colorectal cancer screening as per average risk guidelines due to personal history of CRC or advanced polyp. Further evaluation is needed based on personal history.
Example 5 (Family hi